In [5]:
# Library

import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# n번째 이미지 넣으면 상, 하, 좌, 우 자른 array 리턴해줌

def image_cut_side(image, num, size, pcell):
    p = pcell//4
    x = num%4
    y = num // 4
    image_right = image[p*y:p*(y+1),p*(x+1)-size:p*(x+1),:]
    image_left = image[p*y:p*(y+1),p*x:p*x+size,:]
    image_up = image[p*y:p*y+size,p*x:p*(x+1),:]
    image_down = image[p*(y+1)-size:p*(y+1),p*x:p*(x+1),:]
    # print(image_up.shape)
    # print(image_down.shape)
    # print(image_left.shape)
    # print(image_right.shape)
    return image_up, image_down, image_left, image_right


# 두 array 값을 빼서 제곱한 후 더한 값 리턴해줌
def image_SD(l1,l2):
    r = (l1-l2)**2
    r[r==0]=0.15
    r = r*(np.pi/2)
    return r.reshape(-1).sum()


# 상, 하, 좌, 우 이미지 유사도를 체크해 줌(right 함수는 넣은 이미지의 오른쪽에 올 때 로스를 순서대로 보여줌 - ans에 있는것은 제외하는데 이때 제외된 값의 로스는 100)
def Find_right(image,num,ans,pcell):
    rt = []
    for i in range(16):
        if i not in ans and i != num:
            _,_,_,r = image_cut_side(image,num,1,pcell)
            _,_,l,_ = image_cut_side(image,i,1,pcell)
            rt.append(image_SD(r,l))
        else:
            rt.append(1000000000)
    return rt

def Find_left(image,num,ans,pcell):
    rt = []
    for i in range(16):
        if i not in ans and i != num:
            _,_,l,_ = image_cut_side(image,num,1,pcell)
            _,_,_,r = image_cut_side(image,i,1,pcell)
            rt.append(image_SD(r,l))
        else:
            rt.append(1000000000)
    return rt

def Find_up(image,num,ans,pcell):
    rt = []
    for i in range(16):
        if i not in ans and i != num:
            u,_,_,_ = image_cut_side(image,num,1,pcell)
            _,d,_,_ = image_cut_side(image,i,1,pcell)
            rt.append(image_SD(d,u))
        else:
            rt.append(1000000000)
    return rt

def Find_down(image,num,ans,pcell):
    rt = []
    for i in range(16):
        if i not in ans and i != num:
            _,d,_,_ = image_cut_side(image,num,1,pcell)
            u,_,_,_ = image_cut_side(image,i,1,pcell)
            rt.append(image_SD(d,u))
        else:
            rt.append(1000000000)
    return rt

# ans에 첫 키(왼쪽 위) 하나 넣어주면 나머지 넣어주는 함수(정답과 loss를 리턴해줌)
def Find_ans(image,ans,pcell):
    for i in range(1,5):
        for j in range(1,5):
            loss = 0
            if ans[i][j] == -1:
                sum = np.array([0 for i in range(16)])
                if ans[i-1][j] >= 0:
                    sum = sum + np.array(Find_down(image,ans[i-1][j],ans,pcell))
                if ans[i+1][j] >= 0:
                    sum = sum + np.array(Find_up(image,ans[i+1][j],ans,pcell))
                if ans[i][j-1] >= 0:
                    sum = sum + np.array(Find_right(image,ans[i][j-1],ans,pcell))
                if ans[i][j+1] >= 0:
                    sum = sum + np.array(Find_left(image,ans[i][j+1],ans,pcell))
                ans[i][j] = int(np.where(sum == min(sum))[0][0])
                #if len(np.where(sum == min(sum))[0])>1:
                    #print(min(sum), np.where(sum == min(sum))[0])
                loss += min(sum)
    ans = ans[1:5,1:5].reshape(-1)
    return ans, loss

def IMAGE_fit(image,answer,pcell):
    image_fit = np.zeros((pcell,pcell,3))
    for i,j in zip(range(16),answer):
        image_fit[(pcell//4)*(i//4):(pcell//4)*(i//4+1),(pcell//4)*(i%4):(pcell//4)*(i%4+1),:] = image[(pcell//4)*(j//4):(pcell//4)*(j//4+1),(pcell//4)*(j%4):(pcell//4)*(j%4+1),:]
    return image_fit

def cha(l):
    r = np.zeros(16,dtype=int)
    for i in range(16):
        r[i] = np.where(l==i)[0]
    return r

# submission


submit = pd.read_csv('/home/aiuser/kyunghoney/data/Puzzle/sample_submission.csv')
pcell = 512
from tqdm import tqdm
for i in tqdm(range(submit.shape[0])):
    img = Image.open('/home/aiuser/kyunghoney/data/Puzzle/test/'+submit.ID[i]+".jpg")
    img = img.resize((pcell, pcell))
    img = np.array(img)
    image_scaled = img / 225
    
    answer = np.ones(16)
    loss = 10000
    for a in range(16):
        ans = -1 * np.ones((6,6),dtype='int')
        ans[1][1] = a
        find_ans, find_loss = Find_ans(image_scaled, ans, pcell)
        
        if find_loss < loss:
            answer = find_ans
            loss = find_loss
            
    answer = cha(answer)
    answer += 1
    submit.iloc[i, 1:] = answer
    
submit.to_csv("./save015jh.csv", index=False)

  0%|          | 0/59961 [00:00<?, ?it/s]/tmp/ipykernel_4056520/2685538929.py:110: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  r[i] = np.where(l==i)[0]
100%|██████████| 59961/59961 [1:33:51<00:00, 10.65it/s]


In [2]:
df = pd.read_csv("./save1jh.csv")
df.head()

,Unnamed: 0,ID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,TEST_00000,11,9,6,10,16,3,14,12,8,4,5,15,1,7,2,13
1,1,TEST_00001,6,16,7,2,5,10,13,3,12,8,15,14,4,11,9,1
2,2,TEST_00002,7,2,14,13,6,15,12,16,5,3,4,1,11,9,10,8
3,3,TEST_00003,5,12,11,13,14,4,10,6,2,15,16,9,3,7,8,1
4,4,TEST_00004,16,12,14,1,4,5,15,7,8,13,2,11,3,10,6,9


In [3]:
    
submit.to_csv("./submit1111.csv", index=False)